# MWS Secondary Target Selection

## MWS_CALIB_APOGEE

Maintainers:
  - Ting Li (selection/catalog)
  - Andrew Cooper (secondary target code)

In [1]:
ls ../raw

apogee_xm_gaia_4DESI_v0.fits


{ DESCRIPTION NEEDED }

The selection criteria are:

???

In [2]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_APOGEE/1/code'

In [3]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [4]:
target_class   = 'MWS_CALIB_APOGEE'
input_filename = 'apogee_xm_gaia_4DESI_v0.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [5]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [6]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_APOGEE/1/raw/apogee_xm_gaia_4DESI_v0.fits
Read 264032 rows


In [7]:
print('First 3 rows:')
print(d[0:3])
print(d.colnames)

First 3 rows:
vhelio_avg   verr   vscatter ...       dec             source_id        ebv   
---------- -------- -------- ... ---------------- ------------------- --------
  -98.6882 0.000968      0.0 ... 35.3063874911221 1866727481325405824 0.405349
   14.7794 0.080689 0.337936 ... 35.3474167437634 1866721124773681280 0.351821
  -2.33132 0.008486      0.0 ... 35.3869300615702 1866733700438059776 0.392828
['vhelio_avg', 'verr', 'vscatter', 'obsvhelio_avg', 'obsvscatter', 'obsverr', 'synthvhelio_avg', 'synthvscatter', 'synthverr', 'fe_h', 'j', 'h', 'k', 'field', 'location_id', 'phot_g_mean_mag', 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'pmra', 'pmdec', 'ra', 'dec', 'source_id', 'ebv']


In [8]:
BRIGHT_TIME_BRIGHT_LIMIT = 16.0

print('Mag range: {:5.2f} < G < {:5.2f}'.format(d['phot_g_mean_mag'].min() ,d['phot_g_mean_mag'].max()))
print('Targets brighter than G = 16: {}/{} ({:3.2%})'.format((d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum(),
                                                   len(d),
                                                   (d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum()/len(d)
                                                  ))

bright_time_targets = d['phot_g_mean_mag'] > BRIGHT_TIME_BRIGHT_LIMIT

Mag range:  3.69 < G < 21.26
Targets brighter than G = 16: 237409/264032 (89.92%)


Adjust to secondary target data model:

In [9]:
output = d['ra','dec','source_id'][bright_time_targets]
output.rename_columns(['ra','dec','source_id'],['RA','DEC','REF_ID'])
output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [10]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

In [11]:
output[0:3]

RA,DEC,REF_ID,OVERRIDE,PMRA,PMDEC,REF_CAT,REF_EPOCH
float64,float64,int64,bool,float32,float32,str1,str6
314.957357915112,36.0358351739563,1871266093525759744,False,0.0,0.0,G,2015.5
315.352207996871,36.2025356745186,1871282895438765952,False,0.0,0.0,G,2015.5
314.943058512786,36.906159803936,1871379171422600960,False,0.0,0.0,G,2015.5


Write output:

In [12]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 26623 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_CALIB_APOGEE/1/indata/MWS_CALIB_APOGEE.fits
